In [ ]:
# ============================================================================
# [1] IMPORT LIBRARIES
# ============================================================================
import pandas as pd
import numpy as np
import warnings
import os
import joblib
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

print("Libraries loaded ✓")

Libraries loaded ✓


In [30]:
# ============================================================================
# [2] LOAD DATA
# ============================================================================
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

df_history = pd.read_excel('data/Histori Pemakaian Pelanggan_rev0A..xlsx')
df_temuan = pd.read_excel('data/Temuan_rev0A.xlsx')
df_normal = pd.read_excel('data/Normal.xlsx')

print(f"History : {len(df_history):,} rows")
print(f"Temuan  : {len(df_temuan):,} rows (fraud)")
print(f"Normal  : {len(df_normal):,} rows (normal)")

History : 152,974 rows
Temuan  : 177 rows (fraud)
Normal  : 10,647 rows (normal)


In [108]:
# ============================================================================
# [3] CHECK DUPLICATES
# ============================================================================
# For Temuan & History: Check IE+UE (same customer + same unit)
# For Normal: Only check IE (no UE column)
# Note: Same IE but different UE is NOT a duplicate (same customer, different unit)

print("="*70)
print("DUPLICATE CHECK")
print("="*70)

# TEMUAN: Check IE+UE combination
dup_temuan = df_temuan[df_temuan.duplicated(subset=['IE', 'UE'], keep=False)]
print("\n[TEMUAN] Duplicates (IE+UE):")
print(f"  Total duplicate rows: {len(dup_temuan)}")
if len(dup_temuan) > 0:
    print(f"\n  Duplicate data:")
    print(dup_temuan[['IE', 'UE', 'TARIF', 'DAYA', 'TGL TEMUAN']].sort_values(['IE', 'UE']).to_string(index=False))
else:
    print("  ✓ No duplicates")

# NORMAL: Check IE only (no UE column)
dup_normal = df_normal[df_normal.duplicated(subset=['IE'], keep=False)]
print("\n" + "-"*70)
print("[NORMAL] Duplicates (IE only):")
print(f"  Total duplicate rows: {len(dup_normal)}")
if len(dup_normal) > 0:
    print(f"\n  Duplicate data (showing first 20):")
    print(dup_normal[['IE', 'TARIF', 'DAYA']].sort_values(['IE']).head(20).to_string(index=False))
else:
    print("  ✓ No duplicates")

# HISTORY: Check IE+UE combination
dup_history = df_history[df_history.duplicated(subset=['IE', 'UE'], keep=False)]
print("\n" + "-"*70)
print("[HISTORY] Duplicates (IE+UE):")
print(f"  Total duplicate rows: {len(dup_history)}")
if len(dup_history) > 0:
    print(f"\n  Duplicate data (showing first 20):")
    cols_to_show = ['IE', 'UE', 'TARIF', 'DAYA']
    print(dup_history[cols_to_show].sort_values(['IE', 'UE']).head(20).to_string(index=False))
else:
    print("  ✓ No duplicates")

# Check overlap between Fraud and Normal (IE level)
ie_fraud = set(df_temuan['IE'])
ie_normal = set(df_normal['IE'])
overlap = ie_fraud & ie_normal

print("\n" + "="*70)
print(f"OVERLAP CHECK: Fraud vs Normal (IE level)")
print(f"  IE in both Fraud & Normal: {len(overlap)}")
if len(overlap) > 0:
    print(f"  Note: FRAUD label wins for overlapping IE")
    print(f"  Overlapping IE samples: {list(overlap)[:5]}")
print("="*70)

DUPLICATE CHECK

[TEMUAN] Duplicates (IE+UE):
  Total duplicate rows: 2

  Duplicate data:
              IE       UE TARIF  DAYA TGL TEMUAN
AAAAAwICAgAFBwEE AAAAAwI=    B1  1300 2024-05-22
AAAAAwICAgAFBwEE AAAAAwI=    B1  1300 2025-09-03

----------------------------------------------------------------------
[NORMAL] Duplicates (IE only):
  Total duplicate rows: 1345

  Duplicate data (showing first 20):
              IE TARIF  DAYA
AAAAAAACAgACAQQG    R1  2200
AAAAAAACAgACAQQG    R1  2200
AAAAAAACAgACAwoL    R1   450
AAAAAAACAgACAwoL    R1   450
AAAAAAACAgEACgIA    R1   450
AAAAAAACAgEACgIA    R1   450
AAAAAAACAgICBwME    R1   450
AAAAAAACAgICBwME    R1   450
AAAAAAACAgIDAQQH    B1  1300
AAAAAAACAgIDAQQH    B1  1300
AAAAAAACAgIEBwID    R1   450
AAAAAAACAgIEBwID    R1   450
AAAAAAACAgILAAYC    R1   450
AAAAAAACAgILAAYC    R1   450
AAAAAAACAgMBAAYC   R1M   900
AAAAAAACAgMBAAYC   R1M   900
AAAAAAACAgMCAgMC    B2 13200
AAAAAAACAgMCAgMC    R1   450
AAAAAAACAgMCAgMC    B1  1300
AAAAAAACAgMC

In [110]:
# ============================================================================
# [4] CLEAN DUPLICATES & FILTER UNSUBSCRIBED
# ============================================================================

print("Cleaning duplicates and filtering unsubscribed customers...")

# TEMUAN: Drop duplicates by IE+UE
before_temuan = len(df_temuan)
df_temuan_clean = df_temuan.drop_duplicates(subset=['IE', 'UE'], keep='last')
print(f"\n[TEMUAN]")
print(f"  Before: {before_temuan:,} rows")
print(f"  After : {len(df_temuan_clean):,} rows")
print(f"  Removed: {before_temuan - len(df_temuan_clean):,} duplicates")

# NORMAL: Drop duplicates by IE only
before_normal = len(df_normal)
df_normal_clean = df_normal.drop_duplicates(subset=['IE'], keep='first')
print(f"\n[NORMAL]")
print(f"  Before: {before_normal:,} rows")
print(f"  After : {len(df_normal_clean):,} rows")
print(f"  Removed: {before_normal - len(df_normal_clean):,} duplicates")

# HISTORY: Drop duplicates by IE+UE
before_history = len(df_history)
df_history_clean = df_history.drop_duplicates(subset=['IE', 'UE'], keep='first')
print(f"\n[HISTORY]")
print(f"  Before: {before_history:,} rows")
print(f"  After : {len(df_history_clean):,} rows")
print(f"  Removed: {before_history - len(df_history_clean):,} duplicates")

# Remove overlap from Normal (FRAUD wins)
ie_fraud = set(df_temuan_clean['IE'])
before_overlap = len(df_normal_clean)
df_normal_clean = df_normal_clean[~df_normal_clean['IE'].isin(ie_fraud)]

print(f"\n[OVERLAP HANDLING]")
print(f"  Normal IE overlapping with Fraud: {before_overlap - len(df_normal_clean):,}")
print(f"  Final Normal count: {len(df_normal_clean):,}")

# ============================================================================
# FILTER UNSUBSCRIBED CUSTOMERS (N/A at end = tidak berlangganan lagi)
# ============================================================================
# Extract date columns first
non_date_cols = ['IE', 'UE', 'TARIF', 'DAYA']
date_columns_temp = [col for col in df_history_clean.columns if col not in non_date_cols]
date_columns_temp = sorted(date_columns_temp)

# Check last 3 months - if all NaN, customer unsubscribed
last_3_cols = date_columns_temp[-3:]

# Need to check BEFORE fillna - use original df_history
df_temp = df_history.drop_duplicates(subset=['IE', 'UE'], keep='first')
unsubscribed_mask = df_temp[last_3_cols].isna().all(axis=1)
unsubscribed_count = unsubscribed_mask.sum()

print(f"\n[UNSUBSCRIBED CUSTOMERS]")
print(f"  N/A in last 3 months (unsubscribed): {unsubscribed_count:,}")

# Filter out unsubscribed from df_history_clean
unsubscribed_ie_ue = set(zip(df_temp[unsubscribed_mask]['IE'], df_temp[unsubscribed_mask]['UE']))
before_unsub = len(df_history_clean)

# Create mask for filtering (IE, UE) combination
mask_unsub = df_history_clean.apply(lambda row: (row['IE'], row['UE']) not in unsubscribed_ie_ue, axis=1)
df_history_clean = df_history_clean[mask_unsub]

print(f"  Removed: {before_unsub - len(df_history_clean):,} unsubscribed customers")
print(f"  Remaining: {len(df_history_clean):,} active customers")

print("\n✓ Cleaning complete!")

Cleaning duplicates and filtering unsubscribed customers...

[TEMUAN]
  Before: 177 rows
  After : 176 rows
  Removed: 1 duplicates

[NORMAL]
  Before: 10,647 rows
  After : 9,920 rows
  Removed: 727 duplicates

[HISTORY]
  Before: 152,974 rows
  After : 152,961 rows
  Removed: 13 duplicates

[OVERLAP HANDLING]
  Normal IE overlapping with Fraud: 10
  Final Normal count: 9,910

[UNSUBSCRIBED CUSTOMERS]
  N/A in last 3 months (unsubscribed): 13,904
  Removed: 13,904 unsubscribed customers
  Remaining: 139,057 active customers

✓ Cleaning complete!


In [112]:
# ============================================================================
# [5] LABELING
# ============================================================================
# Label: 1 = Fraud (Temuan), 0 = Normal, -1 = Unknown
ie_fraud = set(df_temuan_clean['IE'])
ie_normal = set(df_normal_clean['IE'])

def assign_label(ie):
    if ie in ie_fraud:
        return 1
    elif ie in ie_normal:
        return 0
    return -1

df_history_clean['label'] = df_history_clean['IE'].apply(assign_label)

label_counts = df_history_clean['label'].value_counts()
print("Labels assigned:")
print(f"  Fraud (1)   : {label_counts.get(1, 0):,}")
print(f"  Normal (0)  : {label_counts.get(0, 0):,}")
print(f"  Unknown (-1): {label_counts.get(-1, 0):,}")

Labels assigned:
  Fraud (1)   : 168
  Normal (0)  : 9,811
  Unknown (-1): 129,078


In [102]:
# ============================================================================
# [5B] VERIFY FRAUD DATA - Check missing fraud cases
# ============================================================================
# Check which fraud IE are NOT in history
ie_temuan_all = set(df_temuan_clean['IE'])
ie_history_all = set(df_history_clean['IE'])

# IE fraud yang tidak ada di history
missing_fraud = ie_temuan_all - ie_history_all
found_fraud = ie_temuan_all & ie_history_all

print("Fraud Verification:")
print(f"  Total unique fraud (Temuan)    : {len(ie_temuan_all):,}")
print(f"  Found in History               : {len(found_fraud):,}")
print(f"  NOT in History (missing)       : {len(missing_fraud):,}")

if len(missing_fraud) > 0:
    print(f"\nMissing fraud IE samples: {list(missing_fraud)[:5]}")

# Check actual labeled fraud in df_history_clean
actual_fraud = (df_history_clean['label'] == 1).sum()
print(f"\nActual fraud labeled in dataset  : {actual_fraud:,}")

# Verify temuan dates - which months have fraud pattern
print("\nTemuan date range:")
print(f"  First: {df_temuan['TGL TEMUAN'].min()}")
print(f"  Last : {df_temuan['TGL TEMUAN'].max()}")

Fraud Verification:
  Total unique fraud (Temuan)    : 176
  Found in History               : 168
  NOT in History (missing)       : 8

Missing fraud IE samples: ['AAAAAgIHAgEDBQoA', 'AAAAAgIHAgIDBwEG', 'AAAAAgIDAgQCAQUE', 'AAAAAgMCBQYCCgAK', 'AAAAAgMCBgYDBgQL']

Actual fraud labeled in dataset  : 168

Temuan date range:
  First: 2024-01-02 00:00:00
  Last : 2025-12-29 00:00:00


In [34]:
# ============================================================================
# [6] DATA INSPECTION
# ============================================================================
print("Column names:")
print(f"  {list(df_history_clean.columns[:5])} ... ({len(df_history_clean.columns)} total)")

print(f"\nData shape: {df_history_clean.shape}")
print(f"\nData types:")
print(df_history_clean.dtypes.value_counts())

print(f"\nMissing values:")
missing = df_history_clean.isnull().sum()
missing_cols = missing[missing > 0]
if len(missing_cols) > 0:
    print(missing_cols)
else:
    print("  No missing values ✓")

Column names:
  ['UE', 'IE', 'TARIF', 'DAYA', datetime.datetime(2021, 3, 1, 0, 0)] ... (64 total)

Data shape: (152948, 64)

Data types:
float64    60
object      3
int64       1
Name: count, dtype: int64

Missing values:
TARIF                  13412
DAYA                   13412
2021-03-01 00:00:00     8944
2021-04-01 00:00:00     8881
2021-05-01 00:00:00     8826
                       ...  
2025-09-01 00:00:00    16270
2025-10-01 00:00:00    15778
2025-11-01 00:00:00    15093
2025-12-01 00:00:00    14724
2026-01-01 00:00:00    14307
Length: 61, dtype: int64


In [111]:
# ============================================================================
# [7] EXTRACT DATE COLUMNS
# ============================================================================
# Date columns already extracted during cleaning, just verify
print(f"Date columns: {len(date_columns_temp)}")
print(f"  First: {date_columns_temp[0]}")
print(f"  Last : {date_columns_temp[-1]}")
print(f"  Range: {len(date_columns_temp)} months")

# Use for rest of analysis
date_columns = date_columns_temp

Date columns: 59
  First: 2021-03-01 00:00:00
  Last : 2026-01-01 00:00:00
  Range: 59 months


In [113]:
# ============================================================================
# [8] HANDLE MISSING VALUES
# ============================================================================
# Fill NaN in usage columns with 0
df_history_clean[date_columns] = df_history_clean[date_columns].fillna(0)

# Check result
remaining_na = df_history_clean.isnull().sum().sum()
print(f"Missing values filled: {remaining_na} remaining")

Missing values filled: 2 remaining


In [60]:
# ============================================================================
# [9] BASIC STATISTICS
# ============================================================================
usage_matrix = df_history_clean[date_columns].values

print("Usage Statistics:")
print(f"  Min  : {usage_matrix.min():.2f}")
print(f"  Max  : {usage_matrix.max():.2f}")
print(f"  Mean : {usage_matrix.mean():.2f}")
print(f"  Std  : {usage_matrix.std():.2f}")

# Statistics by label
for label in [1, 0, -1]:
    label_name = {1: 'Fraud', 0: 'Normal', -1: 'Unknown'}[label]
    mask = df_history_clean['label'] == label
    if mask.sum() > 0:
        values = df_history_clean.loc[mask, date_columns].values
        print(f"\n{label_name}:")
        print(f"  Mean: {values.mean():.2f}")
        print(f"  Std : {values.std():.2f}")

Usage Statistics:
  Min  : 0.00
  Max  : 24229392.00
  Mean : 595.21
  Std  : 20718.42

Fraud:
  Mean: 218.87
  Std : 244.10

Normal:
  Mean: 304.08
  Std : 776.66

Unknown:
  Mean: 617.83
  Std : 21503.14


In [115]:
# ============================================================================
# [11] TARIF & DAYA ANALYSIS (for Visualization)
# ============================================================================
print("="*70)
print("TARIF & DAYA ANALYSIS")
print("="*70)

# TARIF Categories
print("\n[TARIF CATEGORIES]")
tarif_counts = df_history_clean['TARIF'].value_counts().sort_index()
print(f"Total unique tarif: {len(tarif_counts)}")
print(f"\nTarif distribution:")
for tarif, count in tarif_counts.items():
    pct = (count / len(df_history_clean)) * 100
    print(f"  {str(tarif):15s}: {count:>7,} customers ({pct:>5.2f}%)")

# DAYA per TARIF
print("\n" + "="*70)
print("DAYA per TARIF")
print("="*70)

# Sort tarif by converting to string first
tarif_list = df_history_clean['TARIF'].unique()
tarif_list_str = sorted([str(t) for t in tarif_list])

for tarif_str in tarif_list_str:
    # Find original tarif value (could be string or number)
    df_tarif = df_history_clean[df_history_clean['TARIF'].astype(str) == tarif_str]
    daya_values = df_tarif['DAYA'].value_counts().sort_index()
    
    print(f"\n[{tarif_str}] - {len(df_tarif):,} customers")
    print(f"  Unique DAYA values: {len(daya_values)}")
    print(f"  DAYA range: {df_tarif['DAYA'].min()} - {df_tarif['DAYA'].max()}")
    print(f"  Most common DAYA:")
    
    # Show top 5 most common DAYA values
    for daya, count in daya_values.head(5).items():
        pct = (count / len(df_tarif)) * 100
        print(f"    {daya:>6} VA: {count:>6,} customers ({pct:>5.2f}%)")

# Summary statistics
print("\n" + "="*70)
print("SUMMARY")
print("="*70)
print(f"Total TARIF categories: {df_history_clean['TARIF'].nunique()}")
print(f"Total unique DAYA values: {df_history_clean['DAYA'].nunique()}")
print(f"DAYA range (overall): {df_history_clean['DAYA'].min()} - {df_history_clean['DAYA'].max()} VA")

# Breakdown by label
print("\n[BREAKDOWN BY LABEL]")
for label_val in [1, 0, -1]:
    label_name = {1: 'Fraud', 0: 'Normal', -1: 'Unknown'}[label_val]
    df_label = df_history_clean[df_history_clean['label'] == label_val]
    if len(df_label) > 0:
        print(f"\n{label_name} ({len(df_label):,} customers):")
        print(f"  Tarif categories: {df_label['TARIF'].nunique()}")
        print(f"  Top 3 tarif: {', '.join([str(t) for t in df_label['TARIF'].value_counts().head(3).index.tolist()])}")
        print(f"  DAYA range: {df_label['DAYA'].min()} - {df_label['DAYA'].max()} VA")

print("\n" + "="*70)

TARIF & DAYA ANALYSIS

[TARIF CATEGORIES]
Total unique tarif: 18

Tarif distribution:
  B1             :   6,884 customers ( 4.95%)
  B2             :   4,706 customers ( 3.38%)
  B3             :     129 customers ( 0.09%)
  C              :       4 customers ( 0.00%)
  I1             :      26 customers ( 0.02%)
  I2             :      99 customers ( 0.07%)
  I3             :      38 customers ( 0.03%)
  I4             :       1 customers ( 0.00%)
  P1             :     940 customers ( 0.68%)
  P2             :      33 customers ( 0.02%)
  P3             :   2,038 customers ( 1.47%)
  R1             :  71,199 customers (51.20%)
  R1M            :  44,452 customers (31.97%)
  R2             :   3,734 customers ( 2.69%)
  R3             :   1,036 customers ( 0.75%)
  S1             :   3,717 customers ( 2.67%)
  S2             :      15 customers ( 0.01%)
  S2K            :       5 customers ( 0.00%)

DAYA per TARIF

[B1] - 6,884 customers
  Unique DAYA values: 8
  DAYA range: 450.0 - 